## Web scrapping using python

#### References
1. [Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)
2. [Web Scraping using Python](https://www.datacamp.com/community/tutorials/web-scraping-using-python)

In [1]:
# $ python3 -m venv venv
# $ . ./venv/bin/activate

In [2]:
#Better
!pip install requests BeautifulSoup4 fire

  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111027 sha256=948e46bc0c2c458e735a7efdc02d849871d0a5f832228e9a8ecc0792dfb61dbd
  Stored in directory: c:\users\ken\appdata\local\pip\cache\wheels\13\56\13\297fd9814c7ed533d98d22dbd445bb5f3603d41c8dd6a07814
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4007 sha256=5e410fcfde84b08dba4124d689a8d9d8e6a8648e5ed9cb57e4dbb68329627213
  Stored in directory: c:\users\ken\appdata\local\pip\cache\wheels\91\0e\11\1f1321dce76e9c542907008e4a94ff79f8bf525a3fa32b09f3
Successfully built fire termcolor


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

In [3]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_tag_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

In [13]:
res = get_tag_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa','h2')
df = pd.DataFrame(res)
df.head(10)

,0
0,100. Jeffrey Gettleman (@gettleman)
1,99. Africa24 Media (@a24media)
2,98. Scapegoat (@andiMakinana)
3,97. Africa Check (@AfricaCheck)
4,96. James Copnall (@JamesCopnall)
5,95. Online Africa (@oafrica)
6,94. Patrick Ngowi (@PatrickNgowi)
7,93. DOS African Affairs (@StateAfrica)
8,92. MoadowAJE (@Moadow)
9,91. Brendan Boyle (@BrendanSAfrica)


In [80]:

# cleaning the data
df1 = df[0].str.split('.',expand = True)
df1.head(10)
#header = [['position','name','username']]
#df2=pd.DataFrame(header)
#frames= [df2, df1]
#df3=pd.concat(frames)
#print(df3.head(10))
#df4 = df3[1].str.split('(',expand = True)
#df4.head(10)
df2 = df1[1].str.split('(',expand = True)
df2.head(10)
df3 = df2[1].str.split(')',expand = True)
df3.tail(20)
df4 = df3.dropna(axis=0, how='any')
df5 = df4[0].replace('@','', regex=True)
df5.to_csv('influencers.csv')


**Extracting Twitter data**

In [65]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys

import numpy as np

In [7]:

consumer_key = 'OwaWIDBtZ6QOIFjcTH5i6tKPZ'
consumer_secret = 'zsSqmcWmysfz42VgTlw401KmLNAYI2tvEUTW2ISX5vHnyoRHx8'
access_token = '275506568-ASxyT5sfrVXM17QIlGL5j2Ws2Qv2PHIyH4Fc8U0M'
access_token_secret = 'TE8xLOJQFD8uZrqS8xq3akXKy1aOBkVC6uOQ6A52Cvnc4'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = API(auth)


for target in df5:
    print("Getting data for " + target)
    item = api.get_user(target)
    print("name: " + item.name)
    print("screen_name: " + item.screen_name)
    print("description: " + item.description)
    print("statuses_count: " + str(item.statuses_count))
    print("friends_count: " + str(item.friends_count))
    print("followers_count: " + str(item.followers_count))
    

Getting data for gettleman
name: Jeffrey Gettleman
screen_name: gettleman
description: South Asia bureau chief for the New York Times. Author of Love, Africa, a memoir. Winner of the Pulitzer Prize. Retweets do not equal endorsements.
statuses_count: 3766
friends_count: 37
followers_count: 25703
Getting data for a24media
name: A24 Media
screen_name: a24media
description: Africa 24 produces compelling content that makes people want to peek, or gawk, or listen up. Our video creates emotion. Our shows make you think.

@yebo_live
statuses_count: 16852
friends_count: 3060
followers_count: 31292
Getting data for andiMakinana
name: Scapegoat
screen_name: AndiMakinana
description: In pursuit of scoops. I do not write headlines. In an abusive relationship with the mighty and glorious Buccaneers @Orlando_Pirates. @MikeTyson is the greatest.
statuses_count: 142178
friends_count: 2837
followers_count: 101162
Getting data for AfricaCheck
name: Africa Check
screen_name: AfricaCheck
description: Afri

TweepError: [{'message': 'User not found.', 'code': 50}]

In [78]:
targets = pd.DataFrame()

for target in df5:
    try:
        
        print("Getting data for " + target)
        tweets = api.user_timeline(screen_name=target,count=100,include_rts=True)
        user_details = api.get_user(target)
        tweet_list = [tweet for tweet in tweets]
        counter = 0
        favcounter= 0
        #initializing dataframe to store the data
        #targets = pd.DataFrame(columns=['username','popularity score','reach score','relevance score'])
        
        for tweet in tweet_list:
            retweetcount = tweet.retweet_count
            favorite = tweet.favorite_count
            
            counter += retweetcount
            favcounter += favorite
            #print(tweet.text)
            #print('number of retweets ' , retweetcount)
            #print('number of likes ' , favorite)
            popularity_score = counter + favcounter   
            
            
            
        followers_count = user_details.followers_count
        friends_count = user_details.friends_count
        print('number of followers ',followers_count )
        print('number of people',target,'follows', friends_count)
        print('popularity score',popularity_score)
        reachscore = followers_count + friends_count
        print('reach score ', reachscore)
        data = pd.DataFrame(data=[target], columns=['username'])
        data['popularity score'] = np.array([popularity_score])
        data['reach score'] = np.array([reachscore])
        
        targets=pd.concat([targets,data],ignore_index = True)
        
        #targets.append({'username':target, 'popularity score': popularity_score,'reach score':reachscore,'relevance score' : '0'},ignore_index = True)
        
    except tweepy.TweepError :
        pass


targets

Getting data for gettleman
number of followers  25702
number of people gettleman follows 37
popularity score 147100
reach score  25739
Getting data for a24media
number of followers  31294
number of people a24media follows 3060
popularity score 351
reach score  34354
Getting data for andiMakinana
number of followers  101177
number of people andiMakinana follows 2837
popularity score 19349
reach score  104014
Getting data for AfricaCheck
number of followers  68029
number of people AfricaCheck follows 4593
popularity score 945
reach score  72622
Getting data for JamesCopnall
number of followers  21960
number of people JamesCopnall follows 5046
popularity score 125993
reach score  27006
Getting data for oafrica
number of followers  31301
number of people oafrica follows 2879
popularity score 1549
reach score  34180
Getting data for PatrickNgowi
number of followers  22796
number of people PatrickNgowi follows 179
popularity score 7747
reach score  22975
Getting data for StateAfrica
number o

,username,popularity score,reach score
0,gettleman,147100,25739
1,a24media,351,34354
2,andiMakinana,19349,104014
3,AfricaCheck,945,72622
4,JamesCopnall,125993,27006
5,oafrica,1549,34180
6,PatrickNgowi,7747,22975
7,StateAfrica,1,8
8,Moadow,14260,52687
9,BrendanSAfrica,3354,28827


In [79]:
targets.sort_values(['popularity score','reach score'],ascending = [False,False])
targets.to_csv('ken.csv')

In [56]:
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
response = simple_get(url)

In [29]:
res = get_tag_elements(response, search={'find_all':{'class_':'wp-block-embed__wrapper'}})
res

findaing all of {'class_': 'wp-block-embed__wrapper'}


['\nhttps://twitter.com/TsholetsaDomi/status/1238324860536922112\n',
 '\nhttps://twitter.com/Azali_officiel/status/1239649350747332613\n',
 '\n',
 <blockquote class="twitter-tweet" data-dnt="true" data-width="550"><p dir="ltr" lang="en">The Deputy Prime Minister Themba Masuku has today met representatives of the private sector and employees' unions to map a collaborative effort in the fight against <a href="https://twitter.com/hashtag/COVID19?src=hash&amp;ref_src=twsrc%5Etfw">#COVID19</a>. <a href="https://t.co/EIYNGOEKRN">pic.twitter.com/EIYNGOEKRN</a></p>— Eswatini Government (@EswatiniGovern1) <a href="https://twitter.com/EswatiniGovern1/status/1241038139889721346?ref_src=twsrc%5Etfw">March 20, 2020</a></blockquote>,
 <script async="" charset="utf-8" src="https://platform.twitter.com/widgets.js"></script>,
 '\n',
 '\nhttps://twitter.com/SE_Rajoelina/status/1241101811647500288\n',
 '\n',
 <blockquote class="twitter-tweet" data-dnt="true" data-width="550"><p dir="ltr" lang="en">GUIDEL

## Web scrapping using bash script
If the web site has a quite simple HTML, you can easily use curl to perform the request and then extract the needed values using bash commands grep, cut , sed, ..

This tutorial is adapted from [this](https://medium.com/@LiliSousa/web-scraping-with-bash-690e4ee7f98d) medium article

In [ ]:
%%bash 

# curl the page and save content to tmp_file
url = "https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa"
curl -X GET $url -o tmp_file


#!/bin/bash

# write headers to CSV file
echo "Name, twitter_id" >> extractData.csv
n="1"
while [ $n -lt 2 ]
do
  
  #get title
  title=$(cat tmp_file | grep "class=\"twitter-tweet\"" | cut -d ';' -f1 )
  echo $title
  #get author
  #twitter_id=$(cat tmp_file |grep -A1 "class=\"css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0\"" | tail -1)

  #echo "$title, $twitter_id" >> extractData.csv
  #echo "$title, $twitter_id"
    
  n=$[$n+1]

done